In [ ]:
!pip install numpy
!pip install pandas
!pip install mlxtend

In [ ]:
import numpy as np
import os
import pandas as pd
import time

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules

# Input and Data Preprocessing

## Data: transactions

In [ ]:
# input: transactions
df_trans = pd.read_csv('FoodMart-Transactions-1998.csv')

In [ ]:
df_trans.head()

In [ ]:
df_trans["product_id"].value_counts()

In [ ]:
df_trans["customer_id"].value_counts()

### Data Transformation

In [ ]:
trans_dataset = df_trans.groupby(["customer_id","transaction_date"])['product_id'].apply(list)

In [ ]:
trans_dataset

In [ ]:
raw_transactions = list(trans_dataset)

In [ ]:
raw_transactions[:5]

### Transaction encoding for Lib: mlxtend

In [ ]:
te = TransactionEncoder()
te_trans = te.fit(raw_transactions).transform(raw_transactions) #轉為布林值

In [ ]:
print(type(te_trans))
print(te_trans.shape)
te_trans

In [ ]:
# product_id
te.columns_[:10]

## Data: product

In [ ]:
# input: products
df_prod = pd.read_csv('Product-Lookup.csv')

In [ ]:
df_prod.head()

In [ ]:
df_prod.shape

### Filtering

In [ ]:
set(df_prod["product_id"].unique()).difference(set(te.columns_))

In [ ]:
df_prod = df_prod[df_prod["product_id"].isin(te.columns_)]

In [ ]:
df_prod.shape

### Data preprocessing

In [ ]:
df_te = pd.DataFrame(te_trans, columns = df_prod['product_name'])

In [ ]:
print(df_te.shape)
df_te.head()

# Association Analysis

## Threshold setting

In [ ]:
min_sup = 0.00015
min_conf = 0.8

In [ ]:
support_count = min_sup*df_te.shape[0]
print(support_count)

## Frequent pattern/itemset discovering

In [ ]:
start_time = time.time()
frequent_itemsets = fpgrowth(df_te, min_support = min_sup, use_colnames = True)
#frequent_itemsets = fpgrowth(df_te, min_support = min_sup, use_colnames = True)
end_time = time.time()
print(str(end_time - start_time) + " seconds")

In [ ]:
frequent_itemsets.sort_values(by=["support"], ascending = False)

## Association rule discovering

In [ ]:
rules_confidence_10 = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_conf).sort_values(by='confidence', ascending=False)

In [ ]:
rules_confidence_10.head(10)

In [ ]:
rules_lift_10 = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_conf).sort_values(by='lift', ascending=False)

In [ ]:
rules_lift_10.head(10)

# Customer

以下程式碼使用colab內建生成

In [ ]:
df_cust = pd.read_csv('Customer-Lookup.csv')

In [ ]:

# Select relevant customer attributes
customer_attributes = ['customer_state_province', 'yearly_income', 'gender', 'total_children', 'num_children_at_home', 'education', 'occupation', 'homeowner']
df_cust_selected = df_cust[customer_attributes]

df_cust_selected['total_children'] = df_cust_selected['total_children'].apply(lambda x: f'total_children_{x}')
df_cust_selected['num_children_at_home'] = df_cust_selected['num_children_at_home'].apply(lambda x: f'num_children_at_home_{x}')

# Convert homeowner column into "homeowner_yes" and "homeowner_no"
df_cust_selected['homeowner'] = df_cust_selected['homeowner'].apply(lambda x: 'homeowner_yes' if x == 'Y' else 'homeowner_no')

# Convert attributes to strings for transaction encoding
for col in df_cust_selected.columns:
  df_cust_selected[col] = df_cust_selected[col].astype(str)



# Create transactions (each row represents a customer)
transactions = df_cust_selected.values.tolist()

# Transform transactions into a suitable format
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)


# Apply apriori algorithm
frequent_itemsets = apriori(df_encoded, min_support=0.05, use_colnames=True)

# Generate association rules
cust_10_rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.9).sort_values(by='lift', ascending=False).head(10)

cust_10_rules



In [ ]:
import pandas as pd
from google.colab import files

# 假設你的 DataFrame 叫做 df
# 儲存 DataFrame 為 CSV 檔案
cust_10_rules.to_csv('your_data.csv', index=False)

# 下載該檔案到本地
files.download('your_data.csv')

# Product

以下程式碼使用ChatGp4生成

*   提詞: 請寫出每個月的總產品量、交易次數的程式碼，並畫出相對應圖表




In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 確保 transaction_date 是 datetime 格式
df_trans["transaction_date"] = pd.to_datetime(df_trans["transaction_date"])

# 創建月份欄位，使用數字格式的月份
df_trans["month"] = df_trans["transaction_date"].dt.month

# 計算每個月的總產品數量（考慮 quantity）
monthly_product_count = df_trans.groupby("month")["quantity"].sum()

# 計算每個月的交易次數（交易數量）
monthly_transaction_count = df_trans.groupby("month")["transaction_date"].count()

# 整理成 DataFrame
monthly_summary = pd.DataFrame({
    "total_products_sold": monthly_product_count,
    "total_transactions": monthly_transaction_count
})

# 確保按月份排序
monthly_summary = monthly_summary.sort_index()

# 顯示結果
print(monthly_summary)

# 設定 Seaborn 風格
sns.set(style="whitegrid")


plt.figure(figsize=(10, 6))

# 畫出總產品數量
plt.plot(monthly_summary.index, monthly_summary['total_products_sold'], label='Total Products Sold', marker='o', color='b', linestyle='-', linewidth=2)

# 畫出交易次數
plt.plot(monthly_summary.index, monthly_summary['total_transactions'], label='Total Transactions', marker='s', color='g', linestyle='-', linewidth=2)

# 添加標題和標籤
plt.title('Monthly Product Sales and Transaction Counts', fontsize=16)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Count', fontsize=12)

# 顯示圖例
plt.legend()

# 設定 x 軸標籤為月份名稱
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
plt.xticks(monthly_summary.index, month_names)  # 用月份數字索引來獲取對應的名稱

plt.xticks(rotation=45)  # 讓 x 軸標籤更清晰

# 顯示圖形
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 讀取交易數據
df_trans["transaction_date"] = pd.to_datetime(df_trans["transaction_date"])
df_trans["month"] = df_trans["transaction_date"].dt.strftime("%b")  # 轉換為月份 (Jan, Feb, ...)

# 讀取產品對照表
df_products = pd.read_csv("Product-Lookup.csv")

# 確保 product_id 是相同的類型（避免數據類型不同而無法合併）
df_trans["product_id"] = df_trans["product_id"].astype(str)
df_products["product_id"] = df_products["product_id"].astype(str)

df_trans = df_trans.drop(columns=["product_name"], errors="ignore")  # 先刪除可能已存在的 product_name

# 合併產品名稱
df_trans = df_trans.merge(df_products[["product_id", "product_name"]], on="product_id", how="left")

# 計算每個產品每個月的總銷量
monthly_sales = df_trans.groupby(["month", "product_name"])["quantity"].sum().reset_index()

# 找出每個月銷量最高的產品
top_products_df = monthly_sales.loc[monthly_sales.groupby("month")["quantity"].idxmax()]

# 設定月份順序
month_order = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
top_products_df["month"] = pd.Categorical(top_products_df["month"], categories=month_order, ordered=True)

# 確保 `month` 按順序排列
top_products_df = top_products_df.sort_values("month")

# 繪製折線圖（現在用 product_name）
plt.figure(figsize=(12, 6))
sns.lineplot(data=top_products_df, x="month", y="quantity", hue="product_name",marker="o", linewidth=2, color="b")

# 在每個數據點上標示銷量
for index, row in top_products_df.iterrows():
    plt.text(row["month"], row["quantity"], f"{int(row['quantity'])}", ha='center', va='bottom', fontsize=10, fontweight='bold')

# 美化圖表
plt.xlabel("Month", fontsize=12)
plt.ylabel("Total Quantity Sold", fontsize=12)
plt.title("Top-Selling Product in Each Month", fontsize=14)
plt.xticks(rotation=45)
plt.grid(True)
plt.legend(title="Product Name", bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)

# 顯示圖表
plt.show()


In [ ]:
december_sales = df_trans[df_trans['month'] == "Dec"]
june_sales = df_trans[df_trans['month'] == "Jun"]

# 按產品ID統計12月和6月的銷量
december_sales_sum = december_sales.groupby(['product_id', 'product_name'])['quantity'].sum().reset_index()
june_sales_sum = june_sales.groupby(['product_id', 'product_name'])['quantity'].sum().reset_index()

# 合併12月和6月的銷量
sales_comparison = pd.merge(december_sales_sum, june_sales_sum, on=(['product_id', 'product_name']), suffixes=('_december', '_june'))

# 計算12月銷量與6月銷量的比率 (12月銷量 / 6月銷量)
sales_comparison['ratio'] = sales_comparison['quantity_december'] / sales_comparison['quantity_june']

high_difference=sales_comparison.sort_values(by='ratio', ascending=False)

high_difference.head(10)